# Deep learning (visszacsatolt hálók)

A gyakorlaton egy egyszerű, deep learning-alapú nyelvi modellt valósítunk meg, amellyel magyar nyelvű szövegeket dolgozunk fel. Az implementáció alapját egy visszacsatolt neurális háló adja, amelyet a szövegben soron következő karakter predikciójára tanítunk (a korábban megfigyelt karaktereket, mint bemenetet felhasználva). A tanult modellt ezután szöveg generálására használjuk fel.

In [2]:
%pylab inline

import tensorflow as tf
from urllib.request import urlopen

Populating the interactive namespace from numpy and matplotlib


## 1. Bemeneti adatok előállítása

A karaktereket one-hot módon kódoljuk. Minden karaktert egy $\left\lbrace 0,1\right\rbrace^d$ vektorral reprezentálunk, ahol például

\begin{align}
\text{'a'} &= \left[1, 0, 0, \dots, 0 \right], \\
\text{'b'} &= \left[0, 1, 0, \dots, 0 \right], \\
& \vdots \\
\text{'Z'} &= \left[0, 0, 0, \dots, 1 \right], \\
\end{align}

A visszacsatolt hálózat bemenetére egy $\left(n,m,d\right)$ méretű, kimenetére egy $\left(n,d\right)$ méretű tenzor kerül, ahol 

- $n$ a tanítóminták száma
- Minden tanítómintában van a teljes szövegnek egy $m$ hosszú rész-szekvenciája, amelynek minden karakterét az előbb látott $d$-dimenziós vektorokkal kódoljuk (bemenet)
- Minden tanítómintában található egy $d$-dimenziós vektor (kimenet), amely az előbbi rész-szekvencia utáni első rákövetkező karaktert reprezentálja.

A tanítómintákat úgy generáljuk, hogy ezt az $m$ széles ablakot végigtoljuk a teljes szövegen. Összefoglalva, a háló a bemenet-kimenet összefüggés becslése során $m$ lépésnyit tekint vissza.

In [3]:
url = "https://www.mit.bme.hu/system/files/oktatas/targyak/10142/telep.txt" # nevek.txt, telep.txt
seq = urlopen(url).read().decode('utf8')

<b>1.1. feladat.</b> Hozzon létre egy kódoló és dekódoló dictionary-t a one-hot kódoláshoz (azaz előbbiben minden egyedi karakterhez szerepeljen egy egyedi $d$-nél kisebb szám, utóbbi legyen az előbbi inverze).

In [4]:
chars = set(seq)
d     = len(chars)

encoding = dict(zip(chars,range(d)))
decoding = dict(zip(range(d),chars))

<b>1.2. feladat.</b> Alkalmas $m$ választása mellett hozza létre háló bemenetére kerülő $(n,m,d)$ méretű tenzort, valamint a kimenetet reprezentáló $(n,d)$ méretű tenzort!

In [5]:
l = len(seq)
m = 32
w = 2
n = (l-m)//w

chars_in  = np.zeros((n,m,d),dtype=np.bool)
chars_out = np.zeros((n,d),dtype=np.bool)

for i in range(n):
    for j in range(m):
        chars_in[i,j, encoding[seq[i * w + j]]] = 1
    chars_out[i, encoding[seq[i * w + m]]] = 1

## 2. Visszacsatolt háló létrehozása

A hálózathoz tetszőleges architektúrát használhatunk, de ügyelnünk kell arra, hogy a hálózat kimenete kompatibilis legyen az előző lépésben létrehozott kimenettel, azaz a kimenet mérete legyen $d$ (ehhez például illeszthetünk egy teljesen összekötött réteget a visszacsatolt réteg után). Mivel lényegében többosztályos osztályozást végzünk, a veszteségfüggvényt és aktivációs függvényt is ennek megfelelően kell megválasztani. A hálózat létrehozásához itt talál segítséget:

https://keras.io/

(pl. Input, LSTM/GRU/SimpleRNN, Dense rétegek).

<b>2.1. feladat.</b> Hozzon létre a követelményeknek megfelelő neurális hálózatot, majd hozzon létre egy modellt (`tf.keras.Sequential()`).

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(m,d)),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(d)
])
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',loss=loss)

<b>2.2. feladat.</b> Végezze el a tanítást (<i>tf.keras.Model.fit()</i> függvény).

In [21]:
model.fit(chars_in,chars_out,epochs=50,batch_size=256)

Epoch 1/50
59/59 [==============================] - 5s 77ms/step - loss: 2.1263
Epoch 2/50
59/59 [==============================] - 5s 78ms/step - loss: 2.1214
Epoch 3/50
59/59 [==============================] - 4s 74ms/step - loss: 2.1180
Epoch 4/50
59/59 [==============================] - 4s 74ms/step - loss: 2.1132
Epoch 5/50
59/59 [==============================] - 5s 79ms/step - loss: 2.1096
Epoch 6/50
59/59 [==============================] - 5s 79ms/step - loss: 2.1066
Epoch 7/50
59/59 [==============================] - 5s 80ms/step - loss: 2.0992
Epoch 8/50
59/59 [==============================] - 5s 78ms/step - loss: 2.0945
Epoch 9/50
59/59 [==============================] - 5s 76ms/step - loss: 2.0923
Epoch 10/50
59/59 [==============================] - 4s 75ms/step - loss: 2.0872
Epoch 11/50
59/59 [==============================] - 4s 75ms/step - loss: 2.0844
Epoch 12/50
59/59 [==============================] - 4s 74ms/step - loss: 2.0781
Epoch 13/50
59/59 [==================

## 3. Szöveg generálása

A szöveg generálásához a következőképpen járhatunk el:

- Az első bemenetet létrehozhatjuk tetszőlegesen (pl. a szöveg egy részlete)
- Ezen bemenetet a tanult modellre ráadva megkapjuk a rákövetkező karakter eloszlását
- Az eloszlásból mintavételezzük a rákövetkező karaktert, amit ki is írunk
- A bemenetet shifteljük (az utolsó helyre az imént kapott karakter kerül, az első pedig kiesik)

Ezt tetszőleges ideig ismételgetjük.

<b>3.1. feladat.</b> Generáljon 200 karakternyi szöveget a tanult modell felhasználásával. Variálja az architektúrát, dokumentálja a tapasztaltakat.

In [27]:
num_text = 200
final_text = ""

x_pred = np.zeros((1, m, d))
# Véletlenszerű első bemenet
for i in range(32):
  first_inputs = np.zeros(d)
  first_inputs[int(round(rand()*(d-1)))] = 1
  x_pred[0, i, :] = first_inputs

# 1) Distribution 2) sampling the next character  3) shifting
for i in range(num_text):
  pred = model.predict(x_pred)[0]
  pred_index = np.argmax(pred)
  final_text += decoding[pred_index]
  newChar = np.zeros((d))
  newChar[pred_index] = 1
  
  x_pred = np.roll(x_pred, -1, axis=1) #shift
  x_pred[0, m-1, :] = newChar

# Output
print(final_text)

las
Szarasztúr
Kisboros
Szentereszt
Szalatenteresz
Kiskorád
Szentereszt
Szaka
Kisbordony
Szententere
Kiskeresztente
Belesk
Balatanyente
Tárád
Szentereny
Szentereszt
Szalat
Szány
Szentgye
Belyő
Balatat


Először a kapott (1db 64-es LSTM réteg) paraméterekkel próbáltam, magas keresztentrópiát észleltem. A fenti kiemenet két (64-es) réteg melletti eredményt mutat, mely lényegesen jobban teljesít. Egy példát csináltam egy 32-es LSTM-re:




>ala
Kiszalaszenteny
Szalaszenteny
Szerenteny
Szerenteny
Szerenteny
Szerenteny
Szerenteny
Szerenteny
Szerenteny
Szerenteny
Szerenteny
Szerenteny
Szerenteny
Szerenteny
Szerenteny
Szerenteny
Szerenteny
S




^ mint látszik, a felhasználhatóság mércéjét nem igazán üti meg.

<b>3.2. szorgalmi.</b> Hozzon létre és tanítson egy bonyolultabb nyelvi modellt, amely hosszabb szövegek, pl. könyvek generálására is alkalmas.